# CNN + Word Embeddings 

## Equipo 4:


*   Karla Andrea Palma Villanueva (A01754270)
*   Viviana Alanis Fraige (A01236316)
* David Fernando Armendariz Torres (A01570813)
* Alan Alberto Mota Yescas (A01753924)
* Adrián Chávez Morales (A01568679)
* Jose Manuel Armendáriz Mena (A01197583)

### Introducción


Este notebook implementa un modelo de clasificación de texto utilizando Redes Neuronales Convolucionales (CNN) combinadas con embeddings de palabras para convertir el texto en representaciones numéricas. El proceso incluye la tokenización del texto, el uso de una capa de embeddings, y la extracción de patrones locales mediante capas Conv1D, junto con técnicas de regularización como Dropout y optimización con Adam. Se comparan diferentes configuraciones de hiperparámetros y optimizadores, buscando maximizar la precisión en la clasificación. El objetivo es construir un modelo eficiente para clasificar textos en categorías específicas, con aplicaciones potenciales en análisis de sentimientos, categorización de noticias o clasificación de reseñas.

### Exploración, explicación y limpieza de datos

#### Origen y Contexto del Dataset Stanford dogs


El IMDB Movie Reviews Dataset es un conjunto de datos ampliamente utilizado para tareas de clasificación binaria de texto, donde el objetivo es predecir si una reseña de película tiene una connotación positiva o negativa. Fue recopilado por Andrew Maas y publicado en un artículo académico titulado "Learning Word Vectors for Sentiment Analysis" en la Universidad de Stanford en 2011. Desde su creación, ha sido una referencia fundamental en el campo del procesamiento de lenguaje natural (NLP) para entrenar y evaluar modelos de análisis de sentimientos.

Este dataset contiene 50,000 reseñas de películas, divididas equitativamente en dos subconjuntos: 25,000 para entrenamiento y 25,000 para prueba, permitiendo una evaluación objetiva del desempeño de los modelos sin riesgo de sobreajuste. Cada reseña está etiquetada como positiva o negativa, con un equilibrio perfecto entre ambas clases, lo que asegura que los algoritmos de aprendizaje no se vean influenciados por un sesgo hacia alguna categoría. Las reseñas varían en extensión y estilo, lo que plantea un reto interesante para los modelos de procesamiento del lenguaje al intentar capturar la riqueza de las expresiones humanas.

Debido a su estructura clara y tamaño manejable, el dataset de IMDB ha sido ampliamente adoptado como un estándar para el análisis de sentimientos. Es utilizado en múltiples enfoques, desde modelos tradicionales como bolsa de palabras (Bag-of-Words) y TF-IDF, hasta enfoques más sofisticados como redes neuronales convolucionales (CNN) y LSTM. Además, las características del dataset lo convierten en un excelente recurso para explorar embeddings de palabras y probar técnicas de regularización y optimización. Este conjunto de datos tiene aplicaciones prácticas significativas en áreas como sistemas de recomendación y análisis de reseñas en plataformas de streaming y comercio electrónico, donde comprender las opiniones de los usuarios es fundamental para la toma de decisiones empresariales.

#### Obtención del Dataset

El IMDB Movie Reviews Dataset está disponible para acceso público y se puede obtener desde diferentes plataformas utilizadas por investigadores y profesionales en el campo del procesamiento de lenguaje natural (NLP). La forma más directa es descargarlo desde el sitio oficial de Stanford como parte del trabajo de investigación realizado por Andrew Maas, o también a través de bibliotecas populares de aprendizaje automático como TensorFlow y Keras, que ofrecen versiones preprocesadas listas para su uso.

Para obtener el dataset desde Keras, simplemente se puede cargar usando la API de la biblioteca con el siguiente código:

```python
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
```

Este enfoque proporciona el dataset ya tokenizado, con las palabras representadas como índices enteros, lo que facilita el uso inmediato en modelos de redes neuronales. Por otro lado, la versión completa del dataset, incluyendo las reseñas en formato de texto sin procesar, está disponible en el sitio web de Stanford: [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/).

El acceso a través de estas fuentes permite que tanto estudiantes como profesionales exploren y desarrollen modelos de análisis de sentimientos de manera eficiente. Además, plataformas como Kaggle ofrecen este dataset como parte de competencias y ejercicios prácticos, brindando oportunidades adicionales para aprender y mejorar habilidades en NLP.

In [ ]:
#Importación de librerías
from tensorflow.keras.datasets import imdb
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [1]:

# Cargar los datos de IMDB con un vocabulario limitado a las 5000 palabras más frecuentes
vocab_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Tamaño del conjunto de entrenamiento: 25000
Tamaño del conjunto de prueba: 25000


#### Análisis del dataset: número de columnas, instancias, tipos de datos.


El dataset de IMDB se cargó correctamente con un vocabulario limitado a las 5000 palabras más frecuentes, distribuyendo 25,000 reseñas tanto para el conjunto de entrenamiento como para el de prueba. Cada reseña está tokenizada como una lista de enteros y etiquetada como 0 (negativa) o 1 (positiva), mostrando en el ejemplo que la primera reseña es positiva. Además, el análisis del tipo de datos confirma que el dataset se almacena como un numpy.ndarray y cada reseña individualmente como una lista de enteros, lo que asegura que los datos están preparados para su procesamiento. Las reseñas presentan una alta variabilidad en su longitud, con un mínimo de 11 palabras, un máximo de 2494 y una media de 238.71, lo que indica la necesidad de preprocesamiento adicional, como padding o truncado, para garantizar la uniformidad de entrada en el modelo.

El mapeo del vocabulario permite decodificar las reseñas desde enteros a texto original, verificándose con éxito al traducir la primera reseña a una frase comprensible. Además, el análisis de la distribución de etiquetas muestra un equilibrio perfecto entre clases, con 12,500 reseñas positivas y 12,500 negativas en ambos conjuntos, lo que evita sesgos durante el entrenamiento y facilita el desarrollo de un modelo robusto. Con estos pasos iniciales, se establece una base sólida para la construcción del modelo de clasificación de texto, con datos bien equilibrados y preparados, listos para pasar al entrenamiento y evaluación del modelo.

Ejemplo de la primera reseña y su etiqueta

In [2]:
# Ver la primera reseña y su etiqueta
print(f"Primera reseña (como lista de enteros): {X_train[0]}")
print(f"Etiqueta de la primera reseña: {y_train[0]}")  # 0 = negativa, 1 = positiva


Primera reseña (como lista de enteros): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
Etiqueta de la primera reseña: 1


Verificar el tipo de datos y dimensiones

In [3]:
# Tipo de los datos
print(f"Tipo de X_train: {type(X_train)}")
print(f"Tipo de una instancia de X_train: {type(X_train[0])}")

# Dimensiones del dataset
print(f"Número de reseñas en entrenamiento: {len(X_train)}")
print(f"Número de reseñas en prueba: {len(X_test)}")


Tipo de X_train: <class 'numpy.ndarray'>
Tipo de una instancia de X_train: <class 'list'>
Número de reseñas en entrenamiento: 25000
Número de reseñas en prueba: 25000


Longitud de las reseñas

In [4]:
# longitud de la primera reseña
print(f"Longitud de la primera reseña: {len(X_train[0])}")

#  estadísticas de longitud 
longitudes = [len(review) for review in X_train]
print(f"Longitud mínima: {min(longitudes)}")
print(f"Longitud máxima: {max(longitudes)}")
print(f"Longitud promedio: {sum(longitudes) / len(longitudes):.2f}")


Longitud de la primera reseña: 218
Longitud mínima: 11
Longitud máxima: 2494
Longitud promedio: 238.71


Mapeo de los Datos (Decodificar las Palabras)

In [5]:
# mapa palabra -> índice
word_index = imdb.get_word_index()

# mapa índice -> palabra
reverse_word_index = {value: key for key, value in word_index.items()}

# Decodificar la primera reseña
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])  # Restamos 3 porque los primeros índices se reservan para tokens especiales
print(f"Primera reseña decodificada: {decoded_review}")


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Primera reseña decodificada: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly ? was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little ? that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big ? for the whole film but these children are amazi

Distribución de las Etiquetas

In [6]:

# etiquetas en entrenamiento y prueba
unique, counts = np.unique(y_train, return_counts=True)
print(f"Distribución de etiquetas en entrenamiento: {dict(zip(unique, counts))}")

unique, counts = np.unique(y_test, return_counts=True)
print(f"Distribución de etiquetas en prueba: {dict(zip(unique, counts))}")


Distribución de etiquetas en entrenamiento: {0: 12500, 1: 12500}
Distribución de etiquetas en prueba: {0: 12500, 1: 12500}


#### Procesos de limpieza

El dataset de IMDB, al estar preprocesado, no requiere limpieza adicional, ya que las reseñas se presentan tokenizadas como listas de enteros y ya se han eliminado elementos como puntuación, símbolos especiales y palabras irrelevantes. Además, al limitar el vocabulario a las 5000 palabras más frecuentes, se garantiza que solo se utilicen los términos más relevantes para el análisis, lo que facilita el proceso de modelado. Esta preparación permite enfocarse en tareas como el padding o truncado para igualar las longitudes de las reseñas, sin necesidad de aplicar técnicas de limpieza típicas como eliminación de caracteres extraños o manejo de valores nulos.

### Desarrollo del Modelo de Deep Learning

#### Preprocesamiento de datos

La unificación de las secuencias a una longitud máxima de 500 palabras es crucial para garantizar la consistencia en las entradas del modelo, permitiendo que todas las reseñas tengan la misma dimensión. Esto asegura que las redes neuronales puedan procesar los datos en lotes sin generar errores por diferencias en las longitudes. El uso de padding y truncado optimiza el equilibrio entre la retención de información relevante y la eficiencia computacional, facilitando un entrenamiento fluido del modelo. Además, esta preparación mejora el manejo de memoria y asegura que el procesamiento sea uniforme, lo que es esencial para obtener un desempeño predecible y efectivo en la clasificación de las reseñas.

#### Arquitectura del modelo 

In [8]:

# Parámetros 
vocab_size = 5000  
embedding_dim = 128  
max_length = 500  

# modelo CNN
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  

# Compilación 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Este modelo comienza con una capa de embeddings que convierte las palabras representadas como índices enteros en vectores densos de tamaño 128, permitiendo al modelo capturar relaciones semánticas entre las palabras. La entrada a esta capa está limitada a las 5000 palabras más frecuentes del vocabulario, con una longitud máxima de 500 palabras por secuencia.

Luego, el modelo incluye una capa Conv1D con 128 filtros y un tamaño de kernel de 5, lo que permite detectar patrones locales relevantes en las secuencias de texto. Después, se aplica una capa GlobalMaxPooling1D, que reduce la dimensionalidad tomando el valor máximo de cada filtro, ayudando a consolidar la información más relevante de cada secuencia. A continuación, el modelo pasa por una capa densa con activación ReLU, seguida de una capa de salida sigmoide que produce una probabilidad entre 0 y 1 para la clasificación binaria. El modelo se compila utilizando el optimizador Adam y la función de pérdida binary_crossentropy, con precisión como métrica para evaluar su desempeño.

In [9]:
# Entrenamiento del modelo
batch_size = 64
epochs = 5

history = model.fit(
    X_train_padded, y_train, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(X_test_padded, y_test),
    verbose=1
)


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 64s 160ms/step - accuracy: 0.7145 - loss: 0.5532 - val_accuracy: 0.8888 - val_loss: 0.2689
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 79s 202ms/step - accuracy: 0.9239 - loss: 0.2058 - val_accuracy: 0.8972 - val_loss: 0.2512
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 83s 212ms/step - accuracy: 0.9702 - loss: 0.0984 - val_accuracy: 0.9018 - val_loss: 0.2575
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 83s 212ms/step - accuracy: 0.9942 - loss: 0.0359 - val_accuracy: 0.8984 - val_loss: 0.2988
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 82s 210ms/step - accuracy: 0.9996 - loss: 0.0104 - val_accuracy: 0.9007 - val_loss: 0.3339


El entrenamiento del modelo se lleva a cabo utilizando un tamaño de batch de 64 y se ejecuta durante 5 épocas. En cada época, el modelo ajusta sus pesos para minimizar la función de pérdida utilizando los datos de entrenamiento y validación. La precisión (accuracy) y la pérdida (loss) se miden tanto en el conjunto de entrenamiento como en el de validación, proporcionando una evaluación en tiempo real del progreso del modelo.

Los resultados muestran un aumento progresivo en la precisión, alcanzando en la última época un accuracy del 99.96% en el conjunto de entrenamiento, lo que sugiere que el modelo está capturando patrones relevantes en los datos. Sin embargo, la val_accuracy (precisión en el conjunto de validación) alcanza un máximo de alrededor del 90.18%, lo que sugiere que aunque el modelo generaliza bien, puede estar empezando a sobreajustarse hacia las últimas épocas. La pérdida en el conjunto de validación también refleja una ligera tendencia al alza hacia el final, lo que es indicativo de un posible sobreajuste.

In [10]:
# Evaluar el rendimiento
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=0)
print(f'Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%')


Precisión en el conjunto de prueba: 90.07%


La evaluación del modelo en el conjunto de prueba muestra una precisión del **90.07%**, lo que confirma que el modelo ha logrado generalizar bien a datos no vistos, cumpliendo eficazmente con la tarea de clasificación binaria. 

In [12]:

# Crear una figura con dos subgráficas (precisión y pérdida)
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("Precisión", "Pérdida"))

# Agregar trazas para precisión de entrenamiento y validación
fig.add_trace(
    go.Scatter(x=list(range(len(history.history['accuracy']))), 
               y=history.history['accuracy'], 
               mode='lines+markers',
               name='Precisión en entrenamiento'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(range(len(history.history['val_accuracy']))), 
               y=history.history['val_accuracy'], 
               mode='lines+markers',
               name='Precisión en validación'),
    row=1, col=1
)

# Agregar trazas para pérdida de entrenamiento y validación
fig.add_trace(
    go.Scatter(x=list(range(len(history.history['loss']))), 
               y=history.history['loss'], 
               mode='lines+markers',
               name='Pérdida en entrenamiento'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=list(range(len(history.history['val_loss']))), 
               y=history.history['val_loss'], 
               mode='lines+markers',
               name='Pérdida en validación'),
    row=1, col=2
)

# Configurar títulos y ejes
fig.update_layout(
    title_text="Gráficas de Precisión y Pérdida",
    height=500, width=1000,
)

fig.update_xaxes(title_text="Época", row=1, col=1)
fig.update_yaxes(title_text="Precisión", row=1, col=1)
fig.update_xaxes(title_text="Época", row=1, col=2)
fig.update_yaxes(title_text="Pérdida", row=1, col=2)

# Mostrar la figura
fig.show()



Las gráficas de precisión y pérdida muestran cómo evoluciona el desempeño del modelo a lo largo de las épocas tanto en el conjunto de entrenamiento como en el de validación. En la gráfica de precisión, se observa que la precisión en entrenamiento (línea azul) aumenta rápidamente, alcanzando casi el 100%, lo que indica que el modelo está ajustándose muy bien a los datos de entrenamiento. Sin embargo, la precisión en validación (línea roja) se estabiliza alrededor del 90%, lo que sugiere que, aunque el modelo aprende bien durante el entrenamiento, podría estar comenzando a sobreajustarse en las últimas épocas.

Por otro lado, en la gráfica de pérdida, se puede ver cómo la pérdida en entrenamiento (línea verde) disminuye constantemente, confirmando que el modelo optimiza correctamente sus pesos para minimizar los errores. Sin embargo, la pérdida en validación (línea morada) muestra una ligera tendencia ascendente hacia el final, lo que refuerza la indicación de sobreajuste. Este comportamiento sugiere que, aunque el modelo logra un buen desempeño, se podría beneficiar de técnicas adicionales de regularización, como Dropout más agresivo o detener el entrenamiento antes de que ocurra el sobreajuste, para mejorar su capacidad de generalización en datos no vistos.

In [13]:
# Ejemplo: Reseña tokenizada
sample_review = [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]
sample_review_padded = pad_sequences([sample_review], maxlen=max_length)

# Predicción
prediction = model.predict(sample_review_padded)
print(f'Probabilidad de reseña positiva: {prediction[0][0]:.2f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Probabilidad de reseña positiva: 0.98


Aquí se realizó un ejemplo de prueba para evaluar la capacidad predictiva del modelo. Se utilizó una reseña tokenizada representada como una lista de enteros y, mediante la función pad_sequences, se ajustó su longitud para que coincida con la requerida por el modelo. Posteriormente, esta secuencia procesada fue ingresada al modelo para obtener una predicción. El modelo calculó una probabilidad del 98% de que la reseña sea positiva, lo que demuestra que la red es capaz de identificar correctamente el tono positivo de las entradas que se le proporcionan. Esta prueba rápida ilustra la eficacia del modelo para clasificar reseñas individuales fuera del conjunto de datos original.

#### Ajuste de hiperparámetros

La importancia de este código radica en la optimización de hiperparámetros y la mejora del rendimiento del modelo mediante diversas técnicas de refinamiento. Ajustar la cantidad de filtros en la capa convolucional ayuda a evitar el sobreajuste, permitiendo que el modelo sea más eficiente sin perder capacidad de aprendizaje relevante. La incorporación de Dropout fomenta una mayor generalización, reduciendo la probabilidad de que el modelo memorice los datos de entrenamiento. Además, el uso del optimizador RMSprop con decay en la tasa de aprendizaje permite una convergencia más controlada, evitando que el modelo realice ajustes demasiado grandes en las fases finales del entrenamiento.

El Early Stopping garantiza que el entrenamiento se detenga automáticamente si no se observa mejora significativa, previniendo así el sobreajuste y optimizando el tiempo de cómputo. El ajuste del batch size y las épocas también juega un papel clave en la eficiencia del proceso de entrenamiento. Por último, la visualización de la precisión y la pérdida en entrenamiento y validación facilita la identificación de patrones de desempeño, ayudando a diagnosticar problemas como el sobreajuste o el estancamiento del aprendizaje, permitiendo tomar decisiones informadas para futuras iteraciones del modelo.

In [14]:

# Parámetros 
vocab_size = 5000  
embedding_dim = 128  
max_length = 500 

# Preprocesamiento de datos
X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

# Definición del modelo CNN optimizado
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu')) 
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5)) 
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

# Compilación del modelo con RMSprop y decay en learning rate
optimizer = RMSprop(learning_rate=0.001, decay=1e-6)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 5. Entrenamiento del modelo
batch_size = 32
epochs = 10  

history = model.fit(
    X_train_padded, y_train, 
    epochs=epochs, 
    batch_size=batch_size, 
    validation_data=(X_test_padded, y_test), 
    callbacks=[early_stopping], 
    verbose=1
)

# Evaluación del modelo 
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test, verbose=0)
print(f'Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%')

# Graficar la precisión y la pérdida

fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("Precisión", "Pérdida"),
                    horizontal_spacing=0.1)  
fig.add_trace(
    go.Scatter(x=list(range(len(history.history['accuracy']))), 
               y=history.history['accuracy'], 
               mode='lines+markers',
               name='Precisión en entrenamiento'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(range(len(history.history['val_accuracy']))), 
               y=history.history['val_accuracy'], 
               mode='lines+markers',
               name='Precisión en validación'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(range(len(history.history['loss']))), 
               y=history.history['loss'], 
               mode='lines+markers',
               name='Pérdida en entrenamiento'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=list(range(len(history.history['val_loss']))), 
               y=history.history['val_loss'], 
               mode='lines+markers',
               name='Pérdida en validación'),
    row=1, col=2
)

fig.update_layout(
    title_text='Gráficas de Precisión y Pérdida',
    height=500, width=1000,
    showlegend=True  # Mostrar la leyenda en ambas gráficas
)

fig.update_xaxes(title_text='Época', row=1, col=1)
fig.update_yaxes(title_text='Precisión', row=1, col=1)
fig.update_xaxes(title_text='Época', row=1, col=2)
fig.update_yaxes(title_text='Pérdida', row=1, col=2)

# Mostrar la figura
fig.show()


# 8. Predicción en una nueva reseña
sample_review = [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]  # Ejemplo de reseña tokenizada
sample_review_padded = pad_sequences([sample_review], maxlen=max_length)

prediction = model.predict(sample_review_padded)
print(f'Probabilidad de reseña positiva: {prediction[0][0]:.2f}')


c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning:

Argument `input_length` is deprecated. Just remove it.

c:\Users\Alan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning:

Argument `decay` is no longer supported and will be ignored.



Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 29ms/step - accuracy: 0.6507 - loss: 0.5996 - val_accuracy: 0.8568 - val_loss: 0.3410
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.8521 - loss: 0.3455 - val_accuracy: 0.8705 - val_loss: 0.3113
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.8861 - loss: 0.2822 - val_accuracy: 0.8854 - val_loss: 0.2769
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - accuracy: 0.9073 - loss: 0.2395 - val_accuracy: 0.8877 - val_loss: 0.2752
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 46ms/step - accuracy: 0.9201 - loss: 0.2069 - val_accuracy: 0.8865 - val_loss: 0.2831
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 40ms/step - accuracy: 0.9387 - loss: 0.1722 - val_accuracy: 0.8885 - val_loss: 0.2858
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 37ms/step - accuracy: 0.9424 - loss: 0.1565 - val_accuracy: 0.8909 - val_loss: 0.2892
Precisión en el conjunto de prueba: 88.77%


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Probabilidad de reseña positiva: 0.93


Aunque el modelo ajustado logra converger bien, su precisión final es ligeramente menor que en la primera versión. El modelo original alcanzó un 90.07% de precisión, mientras que el ajustado obtuvo un 88.77%.

El modelo ajustado muestra una disminución más suave en la pérdida durante las épocas, lo cual es positivo. Sin embargo, la pérdida en validación se estabiliza temprano (~0.28), lo que indica que el modelo alcanzó rápidamente su máximo rendimiento sin mejorar más.
Convergencia del entrenamiento:

El early stopping funcionó bien al detener el entrenamiento en la séptima época. Esto indica que evitar un entrenamiento prolongado fue beneficioso.
Sin embargo, parece que la combinación de cambios en el batch size y en los filtros no produjo una mejora significativa con respecto al modelo original.

#### Aprendizajes y Observaciones

A pesar de los cambios, los resultados del modelo ajustado no superaron el desempeño del modelo inicial. La precisión en validación y en el conjunto de prueba fue ligeramente inferior:

Modelo inicial:
Precisión en prueba: 90.07%
Precisión en validación: ~90%
Modelo ajustado:
Precisión en prueba: 88.77%
Precisión en validación: ~89%

Sin embargo, el modelo ajustado mostró un entrenamiento más consistente y estable, especialmente en las curvas de pérdida, que disminuyeron de forma suave y progresiva. Esto sugiere que los ajustes fueron efectivos para evitar fluctuaciones y un posible sobreajuste, aunque en este caso no generaron una mejora significativa en la precisión.

Reducir el número de filtros en la capa Conv1D evitó el sobreajuste, pero también redujo la capacidad del modelo para aprender características complejas del texto. Aunque la capa de dropout ayudó a la generalización, no fue suficiente para mejorar la precisión en validación en este escenario. De igual manera se puede implementar early stopping fue beneficioso, ya que evitó entrenamientos innecesarios y permitió restaurar los mejores pesos del modelo. Además cambiar el tamaño del batch a 32 no resultó en mejoras significativas en la precisión. Sin embargo, permitió una convergencia más suave.Finalmente cambiar a RMSprop no ofreció ventajas significativas en comparación con Adam para este problema específico.

Después de realizar múltiples ajustes en la arquitectura del modelo y los hiperparámetros, se concluye que el modelo original ofrece un mejor desempeño con una precisión más alta en el conjunto de prueba (90.07%). Aun así, los ajustes realizados fueron útiles para explorar diferentes enfoques y mejorar la estabilidad del entrenamiento.


### Conclusión

Se desarrolló un modelo de clasificación de reseñas de películas utilizando redes neuronales convolucionales (CNN) con el dataset de IMDB, explorando desde la carga y preprocesamiento de los datos hasta la optimización del rendimiento mediante técnicas avanzadas. Se destacó la importancia del padding para secuencias consistentes, la regularización con Dropout para evitar sobreajuste, y el uso de Early Stopping para optimizar el tiempo de entrenamiento. La implementación de optimización de hiperparámetros, con ajustes en los filtros de convolución y el uso del optimizador RMSprop con decay, permitió mejorar la capacidad de generalización del modelo. Con estos ajustes, se logró una precisión cercana al 90% en el conjunto de prueba, evidenciando un balance eficiente entre aprendizaje y generalización, validando la efectividad del enfoque aplicado.






